**本章内容**

*简单介绍支持向量机*

*利用SMO进行优化*

*利用核函数对数据进行空间转换*

*将SVM和其他分类器进行类比*

**基于最大间隔分隔数据**

优点：*泛化错误率低，计算开销不大， 结果易解释

缺点：*对参数调节和核函数的选择敏感，原始分类器不加修饰仅可用于处理二类问题

适用数据类型：*数值型和标称型数据

## SMO 高效优化算法

工作原理：每次循环中选择两个alpha进行优化处理，一旦找到一对合适的alpha。那么就增大其中一个同时减少另一个。

### 简化版SMO算法 

首先在数据集上遍历每一个alpha， 然后在剩下的alpha集和中随机选择另一个alpha。

需要同时改变两个alpha 由于约束条件alpha·label和为0

伪代码：

创建一个alpha向量并将其初始化为0向量

当迭代次数小于最大迭代次数时（外循环）：

    对数据集中的每个数据向量（内循环）：
    
        如果该数据向量可以被优化：
        
            随机选择另外一个数据向量
            
            同时优化这两个向量
            
            如果两个向量都不能优化，退出内循环
            
    如果所有向量都没被优化，增加迭代次数， 继续下一次循环


In [4]:
import svmMLiA

In [5]:
dataArr, labelArr = svmMLiA.loadDataSet('testSet.txt')

In [6]:
from numpy import *

In [6]:
# 简化版SMO算法

def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    """
    paraments:
        dataMatIn 数据集
        classLabels 类别标签
        C 
        toler 容错率
        maxIter 最大循环次数
        
    returns：
        b
        alphas
    """
    dataMatrix = mat(dataMatIn)
    labelMat = mat(classLabels).transpose()
    b = 0
    m, n = shape(dataMatrix)
    alphas = mat(zeros((m, 1)))
    iter = 0
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m):
            fXi = float(multiply(alphas, labelMat).T*\
                       (dataMatrix*dataMatrix[i, :].T)) + b
            Ei = fXi - float(labelMat[i])
            if ((labelMat[i] * Ei < -toler) and (alphas[i] < C)) or \
                ((labelMat[i] * Ei > toler) and (alphas[i] > 0)):
                j = svmMLiA.selectJrand(i, m) # 随机选择第二个alpha
                fXj = float(multiply(alphas, labelMat).T*\
                           (dataMatrix * dataMatrix[j, :].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                
                # 保证alpha在0到C之间 
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, C + alphas[j] - alphas[i])
                if L == H:
                    print("L == H")
                    continue
                eta = 2.0 * dataMatrix[i, :] * dataMatrix[j, :].T - \
                      dataMatrix[i, :] * dataMatrix[i, :].T - \
                      dataMatrix[j, :] * dataMatrix[j, :].T
                if eta >= 0:
                    print("eta >= 0")
                    continue
                
                alphas[j] -= labelMat[j] * (Ei - Ej) / eta
                alphas[j] = svmMLiA.clipAlpha(alphas[j], H, L)
                if (abs(alphas[j] - alphaIold) < 0.00001):
                    print("j not moving enough")
                    continue
                
                alphas[i] += labelMat[j] * labelMat[i] *\
                            (alphaJold - alphas[j])
                b1 = b - Ei - labelMat[i] * (alphas[i] - alphaIold) *\
                     dataMatrix[i, :] * dataMatrix[i, :].T - \
                     labelMat[j] * (alphas[j] - alphaJold)*\
                     dataMatrix[i, :]*dataMatrix[j, :].T
                b2 = b - Ej - labelMat[i] * (alphas[i] - alphaIold) *\
                     dataMatrix[i,:] * dataMatrix[j, :].T - \
                     labelMat[j] * (alphas[j] - alphaJold) * \
                     dataMatrix[j, :] * dataMatrix[j, :].T
                if (0 < alphas[i]) and (C > alphas[i]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1 + b2) / 2.0
                alphaPairsChanged += 1
                print("iter: %d i: %d, pairs changed %d" %(
                                    iter, i, alphaPairsChanged))
        if (alphaPairsChanged == 0):
            iter += 1
        else:
            iter = 0
        print("iteration number: %d" % iter)
    return b, alphas